In [1]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

2023-04-08 10:04:15.241697: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 10:04:15.330526: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-08 10:04:15.357674: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-08 10:04:15.779161: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Most models are made of layers. The high level implementations of layers and models such as `Keras` and `Sonnet` are built on foundational class `tf.Module`

In [2]:
class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(4.0, name="trainable_variable")
        self.non_trainable_variable = tf.Variable(4.0, trainable=False, name="untrainable_variable")

    def __call__(self, x):
        return self.a_variable * x + self.non_trainable_variable
    
simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

2023-04-08 10:04:17.600008: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-04-08 10:04:17.600033: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ssk-Dell-G15-5511
2023-04-08 10:04:17.600037: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ssk-Dell-G15-5511
2023-04-08 10:04:17.600146: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.85.12
2023-04-08 10:04:17.600159: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.85.12
2023-04-08 10:04:17.600162: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 525.85.12
2023-04-08 10:04:17.600742: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in pe

<tf.Tensor: shape=(), dtype=float32, numpy=24.0>

`__call__` acts like a Python callable and enables us to invoke our models with whatever functions we wish

In [3]:
# Printing all trainable variables
print("Trainable variables: ", simple_module.trainable_variables)

# Printing all non-trainable variables
print("Non-trainable variables: ", simple_module.non_trainable_variables)

# Printing all variables
print("All variables: ", simple_module.variables)

Trainable variables:  (<tf.Variable 'trainable_variable:0' shape=() dtype=float32, numpy=4.0>,)
Non-trainable variables:  (<tf.Variable 'untrainable_variable:0' shape=() dtype=float32, numpy=4.0>,)
All variables:  (<tf.Variable 'trainable_variable:0' shape=() dtype=float32, numpy=4.0>, <tf.Variable 'untrainable_variable:0' shape=() dtype=float32, numpy=4.0>)


Two layer linear layer model with modules.

In [4]:
# Single Dense layer
class Dense(tf.Module):
    def __init__(self, input_features, output_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([input_features, output_features]), name="w")
        self.b = tf.Variable(tf.zeros([output_features]), name="b")

    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

In [5]:
# Complete model with 2 layers
class Sequential(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.Dense1 = Dense(3, 3)
        self.Dense2 = Dense(3, 2)

    def __call__(self, x):
        x = self.Dense1(x)
        return self.Dense2(x)
    
# Create an instance of the model
my_model = Sequential(name="the_model")
print("Model results: ", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results:  tf.Tensor([[5.123894  0.8491444]], shape=(1, 2), dtype=float32)


In [6]:
print("Submodules: ", my_model.submodules)

Submodules:  (<__main__.Dense object at 0x7f14040f2020>, <__main__.Dense object at 0x7f149dd7eb00>)


In [7]:
print("Variables", my_model.variables)

Variables (<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>, <tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.03326015, -1.2288424 ,  1.083338  ],
       [ 1.2849374 ,  1.3659351 ,  0.3940538 ],
       [-0.8969859 , -0.07141123,  0.38183758]], dtype=float32)>, <tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>, <tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.46655902, -0.06010991],
       [-0.5344113 ,  2.2106724 ],
       [ 1.307834  ,  0.1617296 ]], dtype=float32)>)


Creating the same model with flexible input features

In [8]:
class flexibleDense(tf.Module):
    def __init__(self, output_features, name=None):
        super().__init__(name=name)
        self.is_built = False
        self.output_features = output_features

    def __call__(self, x):
        if not self.is_built:
            self.w = tf.Variable(tf.random.normal([x.shape[-1], self.output_features]), name='Weights')
            self.b = tf.Variable(tf.zeros([self.output_features]), name='Bias')
            self.is_built = True
        
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

In [9]:
# Complete model with 2 layers
class Sequential(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.Dense1 = flexibleDense(3)
        self.Dense2 = flexibleDense(2)

    def __call__(self, x):
        x = self.Dense1(x)
        return self.Dense2(x)
    
# Create an instance of the model
my_model = Sequential(name="the_model")
print("Model results: ", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results:  tf.Tensor([[2.6006083 0.7657103]], shape=(1, 2), dtype=float32)


# Saving weights

We can save tf.Module as `checkpoint`and as a `SavedModel`

### Saving Checkpoints

Checkpoints are just the weights, that is, the values of all variables inside the module and its submodules. They consist of two files, the data itself and the metadata.

In [10]:
checkpoint_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(my_model)
checkpoint.write(checkpoint_path)

'my_checkpoint'

In [11]:
!ls my_checkpoint*

my_checkpoint.data-00000-of-00001  my_checkpoint.index


In [12]:
tf.train.list_variables(checkpoint_path)

[('Dense1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('Dense1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('Dense2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('Dense2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2]),
 ('_CHECKPOINTABLE_OBJECT_GRAPH', [])]

In [13]:
# Restoring weights from a checkpoint
new_model = Sequential()
new_checkpoint = tf.train.Checkpoint(new_model)
new_checkpoint.restore("my_checkpoint")

new_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[2.6006083, 0.7657103]], dtype=float32)>

### Saving Functions

TensorFlow can run models without the original Python objects, that is, without the original Python code. This can be done with graphs

In [14]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(3, 3)
    self.dense_2 = Dense(3, 2)

  @tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")

In [15]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]))

tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[0. 0.]
 [0. 0.]], shape=(2, 2), dtype=float32)


Visualize the graph with TensorBoard summary

In [16]:
# Setting up logging
stamp = datetime.now().strftime("%Y%m%d=%H%M%S")
logdir = "logs/func/%s" %stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Else the summary will not see the graph
new_model = Sequential()

# Bracket the function call with tf.summary.trace_on()
# and tf.summary.trace_export()
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function while tracing
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
    tf.summary.trace_export(
        "myfunctrace", step=0, profiler_outdir=logdir
    )

tf.Tensor([[1.9432374 0.       ]], shape=(1, 2), dtype=float32)


2023-04-08 10:04:18.369312: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-08 10:04:18.369333: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.


In [17]:
%tensorboard --logdir logs/func

`SavedModel` contains both a collection of functions and a collection of weights

In [18]:
tf.saved_model.save(my_model, "the_saved_model")

INFO:tensorflow:Assets written to: the_saved_model/assets


In [19]:
ls -l the_saved_model

total 24
drwxr-xr-x 2 ssk ssk  4096 Apr  8 10:04 assets/
-rw-rw-r-- 1 ssk ssk 14296 Apr  8 10:04 saved_model.pb
drwxr-xr-x 2 ssk ssk  4096 Apr  8 10:04 variables/


In [20]:
ls -l the_saved_model/variables

total 8
-rw-rw-r-- 1 ssk ssk 490 Apr  8 10:04 variables.data-00000-of-00001
-rw-rw-r-- 1 ssk ssk 356 Apr  8 10:04 variables.index


Models and layers can be loaded using these without making an instance of the class that created it. This is useeful in situations where we do not have a Python interpreter

In [21]:
new_model = tf.saved_model.load("the_saved_model")

In [22]:
isinstance(new_model, Sequential)

False